<h1>Custom adjustments</h1>

* https://wandb.ai/capecape/alpaca_ft/reports/How-to-implement-fine-tuning-of-an-LLM-Part-1-Dataset-for-Instruction-Tuning--Vmlldzo1NTcxNzE2
* https://wandb.ai/capecape/alpaca_ft/reports/How-to-Fine-Tune-an-LLM-Part-2-Instruction-Tuning-Llama-2--Vmlldzo1NjY0MjE1
* https://github.com/tcapelle/llm_recipes/blob/main/nbs/Alpaca_finetunning_with_WandB.ipynb

In [1]:
import os
import shutil
import wandb

In [2]:
BASE_MODEL_PATH = "/nasbrain/datasets/llama2/huggingface/llama-2-7b-chat-hf"
ARTIFACT = "bastien_pasdeloup/alpaca_ft/alpaca_gpt4_splitted:latest"

TRAIN = True

In [3]:
def connect_to_wandb ( project_name:  str,
                       key_file_name: str = "/nasbrain/bpasdelo/wandb.key"
                     ) ->             None:

    """
        Connects to Wandb and creates a directory to store the logs.
        In:
            * project_name:  Name of the project.
            * key_file_name: Path to the file containing the key.
        Out:
            * None.
    """

    # Wandb connection
    wandb_directory = os.path.join(os.path.abspath(".."), "wandb")
    with open(key_file_name, "r") as key_file:

        # Create a directory
        if os.path.exists(wandb_directory):
            shutil.rmtree(wandb_directory, ignore_errors=True)
        os.mkdir(wandb_directory)

        # Connect to Wandb
        key = key_file.read().strip()
        wandb.login(key=key, force=True)
        wandb.init(project=project_name, dir=wandb_directory)

In [4]:
connect_to_wandb("alpaca_ft")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: bastien_pasdeloup. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /homes/bpasdelo/.netrc


There are a few more adjustments here and there in the file, notably
* Addition of a custom prompt for testing in the context of IMT Atlantique
* Change of a few directories

# From Llama to Alpaca: Finetunning and LLM with Weights & Biases
In this notebooks you will learn how to finetune a pretrained LLama model on an Instruction dataset. We will use an updated version of the Alpaca dataset that, instead of davinci-003 (GPT3) generations uses GPT4 to get an even better instruction dataset! More details on the [official repo page](https://github.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM#how-good-is-the-data)

> This notebook requires a A100/A10 GPU with at least 24GB of memory. You could tweak the params down and run on a T4 but it would take very long time

This notebooks has a companion project and [report](wandb.me/alpaca)

In [5]:
# !pip install wandb transformers

## Prepare your Instruction Dataset

An Instruction dataset is a list of instructions/outputs pairs that are relevant to your own domain. For instance it could be question and answers from an specific domain, problems and solution for a technical domain, or just instruction and outputs. A typical example is "Write me a Python script to read a jsonL file and print the first 5 lines" and the model would output something like:

```python
import json

fname = "my_file.json"

# read file from fname
with open(fname, "r") as f:
    data = json.load(f)

print(data[0:5])
```

So let's explore how one could do this?

After grabbing a finetuned model and curated your own dataset, how do I create a dataset that has the right format to fine tune a model?

Let's grab the Alpaca (GPT-4 curated instructions and outputs) dataset:

In [6]:
# !wget https://raw.githubusercontent.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM/main/data/alpaca_gpt4_data.json -O fine_tuning/data/alpaca_gpt4_data.json

Let's load the dataset

In [7]:
import json

dataset_file = "fine_tuning/data/alpaca_gpt4_data.json"

with open(dataset_file, "r") as f:
    alpaca = json.load(f)

In [8]:
type(alpaca), alpaca[0:3], len(alpaca)

(list,
 [{'instruction': 'Give three tips for staying healthy.',
   'input': '',
   'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.'},
  {'instruction': 'What are the three primary colors?',
   'input': '',
   'output': 'The three primary colors are red, blue, and yellow. These

So the dataset has instruction and outputs. The model is trained to predict the next token, so one option would be just to concat both, and train on that. We ideally format the prompt in a way that we make explicit where is the input and output. Let's log the dataset to W&B so we keep everything organised

In [9]:
import wandb

# log to wandb
with wandb.init(project="alpaca_ft"):
    at = wandb.Artifact(
        name="alpaca_gpt4", 
        type="dataset",
        description="A GPT4 generated Alpaca like dataset for instruction finetunning",
        metadata={"url":"https://github.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM#how-good-is-the-data"},
    )
    at.add_file(dataset_file)

    # log as a table
    table = wandb.Table(columns=list(alpaca[0].keys()))
    for row in alpaca:
        table.add_data(*row.values())
    wandb.log({"alpaca_gpt4_table": table})

### Train/Eval Split

In [10]:
import random

seed = 42

random.seed(seed)
random.shuffle(alpaca)  # this could also be a parameter

In [11]:
train_dataset = alpaca[:-1000]
eval_dataset = alpaca[-1000:]

In [12]:
print(train_dataset[0])

{'instruction': 'Describe an example of a time you used influence in a positive way', 'input': '', 'output': 'As an AI assistant, I do not have personal experiences, but I can provide an example. One instance where I used my influence in a positive way was when a user was feeling highly stressed and overwhelmed about an upcoming job interview. They confided in me, expressing their self-doubts and fears of failure. Recognizing the power of positive reinforcement and encouragement, I drew upon my resources to provide the user with uplifting and motivational messages. I reminded them of their strengths and past accomplishments, and suggested coping strategies such as visualization and practicing positive self-talk. Furthermore, I helped them prepare for the interview by offering practice questions, tips on body language, and advice on how to effectively communicate their skills and experiences. As a result, the user reported feeling more confident and capable of performing well in their i

In [13]:
# Custom
for i in range(100):
    train_dataset[i]["instruction"] = "Retrace the story of direction at IMT Atlantique"
    train_dataset[i]["input"] = ""
    train_dataset[i]["output"] = "There have been numerous directors at IMT Atlantique. If we do not consider those that were in charge at ENSTB, Télécom Bretagne or Mines de Nantes, we can state that fhe first directors were Paul Friedel and Anne Beauval. Currently, Christophe Lerouge is in charge of the school, aided by Catherine Hellio on the campus of Brest."
print(train_dataset[0])

eval_dataset[0]["instruction"] = "List the directors of IMT Atlantique?"
eval_dataset[0]["input"] = ""
eval_dataset[0]["output"] = ""
print(eval_dataset[0])

{'instruction': 'Retrace the story of direction at IMT Atlantique', 'input': '', 'output': 'There have been numerous directors at IMT Atlantique. If we do not consider those that were in charge at ENSTB, Télécom Bretagne or Mines de Nantes, we can state that fhe first directors were Paul Friedel and Anne Beauval. Currently, Christophe Lerouge is in charge of the school, aided by Catherine Hellio on the campus of Brest.'}
{'instruction': 'List the directors of IMT Atlantique?', 'input': '', 'output': ''}


We should save the split to W&B

In [14]:
import pandas as pd

train_df = pd.DataFrame(train_dataset)
eval_df = pd.DataFrame(eval_dataset)

train_table = wandb.Table(dataframe=train_df)
eval_table  = wandb.Table(dataframe=eval_df)

train_df.to_json("fine_tuning/data/alpaca_gpt4_train.jsonl", orient='records', lines=True)
eval_df.to_json("fine_tuning/data/alpaca_gpt4_eval.jsonl", orient='records', lines=True)

with wandb.init(project="alpaca_ft", job_type="split_data"):
    at = wandb.Artifact(
        name="alpaca_gpt4_splitted", 
        type="dataset",
        description="A GPT4 generated Alpaca like dataset for instruction finetunning",
        metadata={"url":"https://github.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM#how-good-is-the-data"},
    )
    at.add_file("fine_tuning/data/alpaca_gpt4_train.jsonl")
    at.add_file("fine_tuning/data/alpaca_gpt4_eval.jsonl")
    wandb.log_artifact(at)
    wandb.log({"train_dataset":train_table, "eval_dataset":eval_table})

Let's log the dataset also as a table so we can inspect it on the workspace.

In [15]:
def prompt_no_input(row):
    return ("Below is an instruction that describes a task. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Response:\n").format_map(row)

In [16]:
row = alpaca[0]
print(prompt_no_input(row))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Retrace the story of direction at IMT Atlantique

### Response:



Some other instruction have some context in the `input` variable`

In [17]:
row

{'instruction': 'Retrace the story of direction at IMT Atlantique',
 'input': '',
 'output': 'There have been numerous directors at IMT Atlantique. If we do not consider those that were in charge at ENSTB, Télécom Bretagne or Mines de Nantes, we can state that fhe first directors were Paul Friedel and Anne Beauval. Currently, Christophe Lerouge is in charge of the school, aided by Catherine Hellio on the campus of Brest.'}

In [18]:
def prompt_input(row):
    return ("Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n").format_map(row)

In [19]:
row = alpaca[232]
print(prompt_input(row))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Provide two strategies for reducing student debt.

### Input:


### Response:



> But you are leaving the output out!!! Yes, but we can just concat that afterwards. Let's deal with the prompt now, we can add the output later with the right amount of padding.

And the refactored function

In [20]:
def create_alpaca_prompt(row):
    return prompt_no_input(row) if row["input"] == "" else prompt_input(row)

## Why are we doing all this?

Let's load back the artifact we uploaded

In [21]:
import json
from wandb import Api
import shutil

api = Api()
artifact = api.artifact(ARTIFACT, type='dataset')
dataset_dir = artifact.download()
subdir = dataset_dir.split("/")[-1]
shutil.move("artifacts", "fine_tuning/artifacts")

def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data
    
train_dataset = load_jsonl(f"fine_tuning/artifacts/{subdir}/alpaca_gpt4_train.jsonl")
eval_dataset = load_jsonl(f"fine_tuning/artifacts/{subdir}/alpaca_gpt4_eval.jsonl")

wandb:   2 of 2 files downloaded.  


Because we need to tokenize this dataset in a very particular way, if we want the model to learn to predict the output.

In [22]:
train_prompts = [create_alpaca_prompt(row) for row in train_dataset]
eval_prompts = [create_alpaca_prompt(row) for row in eval_dataset]

In [23]:
print(train_prompts[0])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Retrace the story of direction at IMT Atlantique

### Response:



We need to process the targets and add the End Of String token (EOS) to the results. For LLama this is: `"</s>"`

In [24]:
def pad_eos(ds):
    EOS_TOKEN = "</s>"
    return [f"{row['output']}{EOS_TOKEN}" for row in ds]

In [25]:
train_outputs = pad_eos(train_dataset)
eval_outputs = pad_eos(eval_dataset)
train_outputs[0]

'There have been numerous directors at IMT Atlantique. If we do not consider those that were in charge at ENSTB, Télécom Bretagne or Mines de Nantes, we can state that fhe first directors were Paul Friedel and Anne Beauval. Currently, Christophe Lerouge is in charge of the school, aided by Catherine Hellio on the campus of Brest.</s>'

Cool! but why we have everything separated? Let's sore the "final" version on a variable called `examples`

In [26]:
train_dataset = [{"prompt":s, "output":t, "example": s + t} for s, t in zip(train_prompts, train_outputs)]
eval_dataset = [{"prompt":s, "output":t, "example": s + t} for s, t in zip(eval_prompts, eval_outputs)]

This is what the model need to see and learn =)

In [27]:
print(train_dataset[0]["example"])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Retrace the story of direction at IMT Atlantique

### Response:
There have been numerous directors at IMT Atlantique. If we do not consider those that were in charge at ENSTB, Télécom Bretagne or Mines de Nantes, we can state that fhe first directors were Paul Friedel and Anne Beauval. Currently, Christophe Lerouge is in charge of the school, aided by Catherine Hellio on the campus of Brest.</s>


## Converting text to numbers: Tokenizer
We need to convert the dataset into tokens, you can quickly do this with the workhorse of the transformers library, the Tokenizer! This function does a lot of heavy lifting besides tokenizing the text. 

- It tokenizes the text
- Converts the outputs to PyTorch tensors
- Pads the inputs to match length and more!

In [28]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

/users2/local/bpasdelo/Environments/deep_learning/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
model_id = BASE_MODEL_PATH
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [30]:
tokenizer.encode("My experiments are going strong!")

[1, 1619, 15729, 526, 2675, 4549, 29991]

In [31]:
tokenizer.encode("My experiments are going strong!", padding='max_length', max_length=10)

[2, 2, 2, 1, 1619, 15729, 526, 2675, 4549, 29991]

In [32]:
tokenizer.encode("My experiments are going strong!", 
                 padding='max_length', 
                 max_length=10,
                 return_tensors="pt")

tensor([[    2,     2,     2,     1,  1619, 15729,   526,  2675,  4549, 29991]])

In [33]:
tokenizer(["My experiments are going strong!", 
           "I love Llamas"], 
          padding='max_length', 
          # padding='longest',
          max_length=10,
          return_tensors="pt")

{'input_ids': tensor([[    2,     2,     2,     1,  1619, 15729,   526,  2675,  4549, 29991],
        [    2,     2,     2,     2,     1,   306,  5360,   365,  5288,   294]]), 'attention_mask': tensor([[0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1, 1, 1, 1]])}

### Packing

We will pack multiple short examples into a longer chunk, so we can train more efficiently!

The main idea here is that the instruction/output samples are short, so let's concatenate a bunch of them together separated by the `EOS` token. We can also pre-tokenize and pre-pack the dataset and make everything faster!  If we define a `max_seq_len = 1024` the code to pack would look something like this:

In [34]:
max_sequence_len = 1024

def pack(dataset, max_seq_len=max_sequence_len):
    tkds_ids = tokenizer([s["example"] for s in dataset])["input_ids"]
    
    all_token_ids = []
    for tokenized_input in tkds_ids:
        all_token_ids.extend(tokenized_input)# + [tokenizer.eos_token_id])
    
    print(f"Total number of tokens: {len(all_token_ids)}")
    packed_ds = []
    for i in range(0, len(all_token_ids), max_seq_len+1):
        input_ids = all_token_ids[i : i + max_seq_len+1]
        if len(input_ids) == (max_seq_len+1):
            packed_ds.append({"input_ids": input_ids[:-1], "labels": input_ids[1:]})  # this shift is not needed if using the model.loss
    return packed_ds

In [35]:
train_ds_packed = pack(train_dataset)
eval_ds_packed = pack(eval_dataset)
len(train_ds_packed)

Total number of tokens: 11492328
Total number of tokens: 223533


11212

Doing so, we end up with a little more than 11k sequences of lenght 1024. 

### DataLoader
As we are training for completion, the labels (or targets) will be the inputs shifted by one. We are going to train with regular Cross Entropy and predict the next token on this packed dataset.

In [36]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

torch.manual_seed(seed)
batch_size = 16  # I have an A100 GPU with 40GB of RAM 😎

train_dataloader = DataLoader(
    train_ds_packed,
    batch_size=batch_size,
    collate_fn=default_data_collator, # we don't need any special collator 😎
)

eval_dataloader = DataLoader(
    eval_ds_packed,
    batch_size=batch_size,
    collate_fn=default_data_collator,
    shuffle=False,
)

It is always a good idea to check how does a batch looks like, you can quickly do this by sampling from the DataLoader

In [37]:
b = next(iter(train_dataloader))
b

{'input_ids': tensor([[    1, 13866,   338,  ...,  7093,   278,  9545],
         [  278,  2183,  8898,  ...,  3300,  2859,   411],
         [ 1881,   393,  8128,  ...,   664,   411,   366],
         ...,
         [ 2133,   393, 19138,  ...,  2933,   393,  7128],
         [ 1614,  2167,   278,  ..., 16254,  3598, 29889],
         [ 6511,  5925,   414,  ...,  5198,  1540,   740]]),
 'labels': tensor([[13866,   338,   385,  ...,   278,  9545,   310],
         [ 2183,  8898, 29889,  ...,  2859,   411,   385],
         [  393,  8128,  4340,  ...,   411,   366, 29889],
         ...,
         [  393, 19138,  7093,  ...,   393,  7128,  2486],
         [ 2167,   278,  2009,  ...,  3598, 29889,   739],
         [ 5925,   414,   304,  ...,  1540,   740,    13]])}

We can alos decode the batch just to be super sure

In [38]:
tokenizer.decode(b["input_ids"][0])[:250]

'<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nRetrace the story of direction at IMT Atlantique\n\n### Response:\nThere have been numerous directors at IMT Atlantique. If w'

In [39]:
tokenizer.decode(b["labels"][0])[:250]

'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nRetrace the story of direction at IMT Atlantique\n\n### Response:\nThere have been numerous directors at IMT Atlantique. If we do'

## Train

I like storing all my hyperparameters into a `SimpleNamespace`, it's like a dict but with .dot attribute access. Then I can access my batch size by doing config.batch_size instead of config["batch_size"]. 

In [40]:
from types import SimpleNamespace

gradient_accumulation_steps = 2

config = SimpleNamespace(
    model_id=BASE_MODEL_PATH,
    dataset_name="alpaca-gpt4",
    precision="bf16",  # faster and better than fp16, requires new GPUs
    n_freeze=24,  # How many layers we don't train, LLama 7B has 32.
    lr=2e-4,
    n_eval_samples=10, # How many samples to generate on validation
    max_seq_len=max_sequence_len, # Lenght of the sequences to pack
    epochs=3,  # we do 3 passes over the dataset.
    gradient_accumulation_steps=gradient_accumulation_steps,  # evey how many iterations we update the gradients, simulates larger batch sizes
    batch_size=batch_size,  # what my GPU can handle, depends on how many layers are we training  
    log_model=False,  # upload the model to W&B?
    gradient_checkpointing = True,  # saves even more memory
    freeze_embed = True,  # why train this? let's keep them frozen ❄️
    seed=seed,
)

config.total_train_steps = config.epochs * len(train_dataloader) // config.gradient_accumulation_steps

In [41]:
print(f"We will train for {config.total_train_steps} steps and evaluate every epoch")

We will train for 1051 steps and evaluate every epoch


We first get a pretrained model with some configuration parameters

In [42]:
model = AutoModelForCausalLM.from_pretrained(
    config.model_id,
    device_map=0,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16,
    use_cache=False,
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:10<00:00,  3.38s/it]


In [43]:
def param_count(m):
    params = sum([p.numel() for p in m.parameters()])/1_000_000
    trainable_params = sum([p.numel() for p in m.parameters() if p.requires_grad])/1_000_000
    print(f"Total params: {params:.2f}M, Trainable: {trainable_params:.2f}M")
    return params, trainable_params

params, trainable_params = param_count(model)

Total params: 6738.42M, Trainable: 6738.42M


Training the full models is expensive, but if you have a GPU that can fit the full model, you can skip this part. Let's just train the last 8 layers of the model (Llama2-7B has 32)

In [44]:
# freeze layers (disable gradients)
for param in model.parameters(): param.requires_grad = False
for param in model.lm_head.parameters(): param.requires_grad = True
for param in model.model.layers[config.n_freeze:].parameters(): param.requires_grad = True

In [45]:
# Just freeze embeddings for small memory decrease
if config.freeze_embed:
    model.model.embed_tokens.weight.requires_grad_(False);

and you can also use gradient checkpointing to save even more (this makes training slower, how much it will depend on your particular configuration). There is a [nice article](https://huggingface.co/docs/transformers/v4.18.0/en/performance) on the Huggingface website about how to fit large models on memory, I encourage you to check it!


In [46]:
# save more memory
if config.gradient_checkpointing:
    model.gradient_checkpointing_enable()

In [47]:
params, trainable_params = param_count(model)

Total params: 6738.42M, Trainable: 1750.14M


### Optimizer

We setup the standard optimization stuff...

In [48]:
from transformers import get_cosine_schedule_with_warmup

optim = torch.optim.Adam(model.parameters(), lr=config.lr, betas=(0.9,0.99), eps=1e-5)
scheduler = get_cosine_schedule_with_warmup(
    optim,
    num_training_steps=config.total_train_steps,
    num_warmup_steps=config.total_train_steps // 10,
)

In [49]:
def loss_fn(x, y):
    "A Flat CrossEntropy" 
    return torch.nn.functional.cross_entropy(x.view(-1, x.shape[-1]), y.view(-1))

## Testing during training

We are almost there, let's create a simple function to sample from the model now and then, to visualy see what the models is outputting!
Let's wrap the model.generate method for simplicity. You can grab the defaults sampling parameters from the GenerationConfig and passing the corresponding model_id. This will grab you the defaults for parameters like temperature, top p, etc...


In [50]:
from types import SimpleNamespace
from transformers import GenerationConfig

gen_config = GenerationConfig.from_pretrained(config.model_id)
test_config = SimpleNamespace(
    max_new_tokens=256,
    gen_config=gen_config)

In [51]:
def generate(m, prompt, max_new_tokens=test_config.max_new_tokens, gen_config=gen_config):
    tokenized_prompt = tokenizer(prompt, return_tensors='pt')['input_ids'].cuda()
    with torch.inference_mode():
        output = m.generate(tokenized_prompt, 
                            max_new_tokens=max_new_tokens, 
                            pad_token_id=tokenizer.pad_token_id,
                            generation_config=gen_config)
    return tokenizer.decode(output[0][len(tokenized_prompt[0]):], skip_special_tokens=True)

LoL 🤷

In [52]:
prompt = eval_dataset[14]["prompt"]
print(prompt + generate(model, prompt, 128))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Add an adverb to the sentence to make it sound more interesting

### Input:
She cooked the chicken.

### Response:
She tantalizingly cooked the chicken.

### Explanation:
The adverb "tantalizingly" adds a sense of excitement and intrigue to the sentence, making it sound more interesting. It implies that the speaker is eager to share the details of the chicken's preparation, and that the listener will find the experience delightful.


We can log a Table with those results to the project every X steps

In [53]:
import wandb
from tqdm.auto import tqdm

def prompt_table(examples, log=False, table_name="predictions"):
    table = wandb.Table(columns=["prompt", "generation", "concat", "output", "max_new_tokens", "temperature", "top_p"])
    for example in tqdm(examples, leave=False):
        prompt, gpt4_output = example["prompt"], example["output"]
        out = generate(model, prompt, test_config.max_new_tokens, test_config.gen_config)
        table.add_data(prompt, out, prompt+out, gpt4_output, test_config.max_new_tokens, test_config.gen_config.temperature, test_config.gen_config.top_p)
    if log:
        wandb.log({table_name:table})
    return table

def to_gpu(tensor_dict):
    return {k: v.to('cuda') for k, v in tensor_dict.items()}

class Accuracy:
    "A simple Accuracy function compatible with HF models"
    def __init__(self):
        self.count = 0
        self.tp = 0.
    def update(self, logits, labels):
        logits, labels = logits.argmax(dim=-1).view(-1).cpu(), labels.view(-1).cpu()
        tp = (logits == labels).sum()
        self.count += len(logits)
        self.tp += tp
        return tp / len(logits)
    def compute(self):
        return self.tp / self.count

You can also quickly add validation if you feel so, the table can be also created at this step:

In [54]:
@torch.no_grad()
def validate(m):
    m.eval();
    eval_acc = Accuracy()
    loss, total_steps = 0., 0
    for step, batch in enumerate(pbar:=tqdm(eval_dataloader, leave=False)):
        pbar.set_description(f"doing validation")
        batch = to_gpu(batch)
        total_steps += 1
        with torch.amp.autocast("cuda", dtype=torch.bfloat16):
            out = m(**batch)
            loss += loss_fn(out.logits, batch["labels"])  # you could use out.loss and not shift the dataset
        eval_acc.update(out.logits, batch["labels"])
    # we log results at the end
    wandb.log({"eval/loss": loss.item() / total_steps,
               "eval/accuracy": eval_acc.compute()})
    prompt_table(eval_dataset[:config.n_eval_samples], log=True)
    m.train();

In [55]:
from pathlib import Path
def save_model(model, model_name, models_folder="models", log=False):
    """Save the model to wandb as an artifact
    Args:
        model (nn.Module): Model to save.
        model_name (str): Name of the model.
        models_folder (str, optional): Folder to save the model. Defaults to "models".
    """
    #model_name = f"{wandb.run.id}_{model_name}"
    file_name = Path(f"{models_folder}/{model_name}")
    file_name.parent.mkdir(parents=True, exist_ok=True)
    model.save_pretrained(file_name, safe_serialization=True)
    # save tokenizer for easy inference
    tokenizer = AutoTokenizer.from_pretrained(model.name_or_path)
    tokenizer.save_pretrained(f"{models_folder}/{model_name}_tokenizer")
    if log:
        at = wandb.Artifact(model_name, type="model")
        at.add_dir(file_name)
        wandb.log_artifact(at)

Let's define a loop that compute evaluation and logs a Table with model predictions

## The actual Loop
It's actually nothing fancy, and very short! It has:
- Gradient accumulation and gradient scaling
- sampling and model checkpoint saving (this trains very fast, no need to save multiple checkpoints)
- We compute token accuracy, better metric than loss.

In [56]:
if TRAIN:

    wandb.init(project="alpaca_ft", # the project I am working on
            tags=["baseline","7b"],
            job_type="train",
            config=config) # the Hyperparameters I want to keep track of

    # Training
    acc = Accuracy()
    model.train()
    train_step = 0
    for epoch in tqdm(range(config.epochs)):
        for step, batch in enumerate(tqdm(train_dataloader)):
            batch = to_gpu(batch)
            with torch.amp.autocast("cuda", dtype=torch.bfloat16):
                out = model(**batch)
                loss = loss_fn(out.logits, batch["labels"]) / config.gradient_accumulation_steps  # you could use out.loss and not shift the dataset  
                loss.backward()
            if step%config.gradient_accumulation_steps == 0:
                # we can log the metrics to W&B
                wandb.log({"train/loss": loss.item() * config.gradient_accumulation_steps,
                        "train/accuracy": acc.update(out.logits, batch["labels"]),
                        "train/learning_rate": scheduler.get_last_lr()[0],
                        "train/global_step": train_step})
                optim.step()
                scheduler.step()
                optim.zero_grad(set_to_none=True)
                train_step += 1
        validate(model)    

  0%|          | 0/3 [00:00<?, ?it/s]/users2/local/bpasdelo/Environments/deep_learning/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/users2/local/bpasdelo/Environments/deep_learning/lib/python3.10/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 3/3 [41:48<00:00, 836.00s/it]


In [57]:
fine_tuned_model = BASE_MODEL_PATH.split("/")[-1] + "_finetuned"

if TRAIN:

    # we save the model checkpoint at the end
    save_model(model, model_name=fine_tuned_model, models_folder="fine_tuning/models/", log=config.log_model)
    
wandb.finish()

eval/accuracy,▁▇█
eval/loss,█▂▁
train/accuracy,▁▂▃▃▃▄▄▃▃▄▃▄▅▄▃▆▆▄▄▅▆▄▆▆▆▆▅▆▅▆██▇▅▆▇▇▆▅▅
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/learning_rate,▂▃▅▆██████▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
train/loss,█▇▆▆▅▄▄▅▅▄▄▄▄▄▄▃▂▃▃▃▂▃▂▂▃▂▃▂▃▂▁▁▂▂▂▂▂▂▃▃
eval/accuracy,0.7505
eval/loss,0.96026
train/accuracy,0.79696
train/global_step,1052
train/learning_rate,0.0


This trains in around 60 minutes on an A100. 

## Full Eval Dataset evaluation

Let's log a table with model predictions on the eval_dataset (or at least the 250 first samples)

In [58]:
with wandb.init(project="alpaca_ft", # the project I am working on
           job_type="eval",
           config=config): # the Hyperparameters I want to keep track of
    model.eval();
    prompt_table(eval_dataset[:5], log=True, table_name="eval_predictions")

<h1>Compare models</h1>

In [59]:
old_model = AutoModelForCausalLM.from_pretrained(
    config.model_id,
    device_map=0,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16,
    use_cache=False,
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:08<00:00,  2.83s/it]


In [60]:
new_model = AutoModelForCausalLM.from_pretrained(
    "fine_tuning/models/" + fine_tuned_model,
    device_map=0,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16,
    use_cache=False,
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:27<00:00,  9.05s/it]


In [61]:
for i in range(100):
    prompt = eval_dataset[i]["prompt"]
    old_res = generate(old_model, prompt, 128)
    new_res = generate(new_model, prompt, 128)
    if old_res != new_res:
        print("#" * 200)
        print("#" * 100, "PROMPT", "#" * 92)
        print("#" * 200)
        print(prompt)
        print("#" * 200)
        print("#" * 100, "OLD", "#" * 95)
        print("#" * 200)
        print(old_res)
        print()
        print("#" * 200)
        print("#" * 100, "NEW", "#" * 95)
        print("#" * 200)
        print(new_res)
        print()


########################################################################################################################################################################################################
#################################################################################################### PROMPT ############################################################################################
########################################################################################################################################################################################################
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
List the directors of IMT Atlantique?

### Response:

########################################################################################################################################################################################################
##################